In [1]:
from sqlalchemy import create_engine,text
import pandas as pd
import urllib

In [3]:
import pyodbc
print(pyodbc.drivers())

['SQL Server', 'SQL Server Native Client 11.0', 'SQL Server Native Client RDA 11.0', 'ODBC Driver 17 for SQL Server', 'Microsoft Access Driver (*.mdb, *.accdb)', 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)', 'Microsoft Access Text Driver (*.txt, *.csv)', 'Microsoft Access dBASE Driver (*.dbf, *.ndx, *.mdx)']


In [4]:

server = ".\\SQLEXPRESS"
database = "sales-analytics"

params = urllib.parse.quote_plus(
    f"DRIVER=ODBC Driver 17 for SQL Server;"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"Trusted_Connection=yes;"
)

engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

# Test connection
with engine.connect() as conn:
    result = conn.execute(text("SELECT 1"))
    print(result.scalar())


1


In [5]:


orders = pd.read_csv(
    "D:/DA project/sales-analytics-project/data/clean/orders_clean.csv",
    parse_dates=[
        "order_purchase_timestamp",
        "order_approved_at",
        "order_delivered_carrier_date",
        "order_delivered_customer_date",
        "order_estimated_delivery_date"
    ]
)

print("Rows in orders CSV:", len(orders))
orders.head(2)


Rows in orders CSV: 99441


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,has_approved_date,has_carrier_date,has_delivery_date,is_canceled
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,True,True,True,False
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,True,True,True,False


In [6]:
orders.dtypes

order_id                                 object
customer_id                              object
order_status                             object
order_purchase_timestamp         datetime64[ns]
order_approved_at                datetime64[ns]
order_delivered_carrier_date     datetime64[ns]
order_delivered_customer_date    datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
has_approved_date                          bool
has_carrier_date                           bool
has_delivery_date                          bool
is_canceled                                bool
dtype: object

In [7]:
orders["order_purchase_date"] = orders["order_purchase_timestamp"].dt.date


In [8]:
fact_orders_cols = [
    "order_id",
    "customer_id",
    "order_status",
    "order_purchase_date",
    "order_purchase_timestamp",
    "order_approved_at",
    "order_delivered_carrier_date",
    "order_delivered_customer_date",
    "order_estimated_delivery_date",
    "has_approved_date",
    "has_carrier_date",
    "has_delivery_date",
    "is_canceled"
]

fact_orders_df = orders[fact_orders_cols]


In [9]:
fact_orders_df.isnull().sum()


order_id                            0
customer_id                         0
order_status                        0
order_purchase_date                 0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
has_approved_date                   0
has_carrier_date                    0
has_delivery_date                   0
is_canceled                         0
dtype: int64

In [10]:
fact_orders_df.to_sql(
    name="fact_orders",
    con=engine,
    schema="dbo",
    if_exists="append",
    index=False,
    chunksize=5000
)


265